# 神经网络
神经网络可以用 <font color=red>torch.nn</font> 这个包来构建

我们已经介绍过 <font color=red>autograd</font>，而 <font color=red>nn</font> 是依赖于 <font color=red>nn</font> 来定义模型以及求微分。一个 <font color=red>nn.Module</font> 包含构建模型的 <font color=red>层（layers）</font> 和一个方法 <font color=red>forward（input）</font>。<font color=red>forward（input）</font> 这个方法返回 <font color=red>output</font>。

训练一个神经网络的典型步骤如下：
1. 定义一个神经网络。这个网络有一些可以学习的参数（或者权重）
2. 遍历输入进行迭代
3. 通过神经网络处理输入
4. 计算损失（离正确的输出还有多远）
5. 对神经网络的参数进行梯度反向传播
6. 更新神经网络的权重，一般用简单的规则：<font color=red>weight = weight - learning_rate * gradient</font>

## 定义神经网络
让我们来定义一个神经网络：

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


我们已经定义了 <font color=red>forward</font> 方法。当你使用了 <font color=red>autograd</font> 的话， <font color=red>backward</font> 方法（用于梯度的计算）会被自动定义。我们可以在 <font color=red>forward</font> 方法里用各种操作（operations）。

模型中那些可学习的参数可以通过调用 <font color=red>net.parameters()</font> 方法返回：

In [13]:
print(net.parameters()) # we can see the net.parameters() method is return a generator
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight

<generator object Module.parameters at 0x10cd10728>
10
torch.Size([6, 1, 5, 5])


In [14]:
# The net has 5 layers, but len(params) is 10. Because there are bias at every layers
for i in range(10):
    print("the {}th params size is: ".format(params[i].size()))

the torch.Size([6, 1, 5, 5])th params size is: 
the torch.Size([6])th params size is: 
the torch.Size([16, 6, 5, 5])th params size is: 
the torch.Size([16])th params size is: 
the torch.Size([120, 400])th params size is: 
the torch.Size([120])th params size is: 
the torch.Size([84, 120])th params size is: 
the torch.Size([84])th params size is: 
the torch.Size([10, 84])th params size is: 
the torch.Size([10])th params size is: 


现在我们来尝试一个32x32的输入。注意对于这个网络（LeNet）而言，其输入的大小必须是32x32。如果想把这个网络用于 MNIST 数据集，请把数据集中的图片转变为32x32大小

In [15]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0066, -0.0071, -0.0008, -0.0798,  0.0594,  0.1498, -0.0205,
          0.0077, -0.0582,  0.1156]])


把所有参数的梯度缓冲区（buffers）清零，然后用随机梯度进行反向传播：

In [16]:
net.zero_grad()
out.backward(torch.randn(1, 10))

<font color=red>**注意**：</font> <font color=red>torch.nn</font> 只支持 mini-batches。整个 torch.nn 包只支持是 mini-batch 的样本，而不支持单个样本。
例如：<font color=red>nn.Conv2d</font> 的输入是个 4D 的 Tensor <font color=red>nSamples * nChanels * Height * Weight</font>。
如果输入为单个样本的话，只需要用 <font color=red>input.unsqueeze(0)</font> 来多增加一维假的 batch 维度。

在继续介绍之前，让我们来回顾下截止目前已经见过的 classes。
### 复习：
<font color=red>torch.Tensor</font> - 一个支持像 <font color=red>backward（）</font> 这样的自动求取梯度操作的多维数组。而且也存放着相对于该 tensor 的梯度。
<font color=red>nn.Module</font> - 神经网络模块。是一种非常方便的用于封装参数的方法。它提供了将参数移入GPU，载入和转出等等功能。
<font color=red>nn.Parameter</font> - 是一种 Tensor。可以通过赋值来确定一个 <font color=red>Module</font> 的属性。
<font color=red>autograd.Function</font> - 用于实现 autograd 操作中 forward 和 backward 的定义。每个 <font color=red>autograd.Function</font> - 用于实现 autograd 操作中 forward 和 backward 的定义。每个 <font color=red>Tensor</font> 操作创建了至少一个 <font color=red>Function</font> 节点（node）。这个节点和产生该 Tensor 的 functions 连接并编码它的历史。

### 截止目前，我们已经介绍了：
定义一个神经网络
前向处理输入和调用反向传播
### 还剩下：
计算损失（loss）
更新神经网络的权重

## 损失函数（Loss Function）
损失函数的接受一对输入（输出【output】和目标【target】）得到一个输出值。这个输出值估计了模型 output 和 target 之间的距离。（我们希望loss越小越好，即 output 和 target 越近越好）

nn 包中有很多种不同的 <font color=red>loss function</font>。一个简单的损失函数是： <font color=red>nn.MSELoss</font>。它计算输出和目标之间的均方差（mean-squared error)。

例如：

In [17]:
output = net(input)
target = torch.randn(10) # a dummy target, for example
target = target.view(1, -1) # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(38.2243)


现在，如果你顺着 <font color=red>loss</font> 方向传播的方向用它的 <font color=red>.grad_fn</font> 属性，你将看到如下的一幅计算图：
```
input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
      -> view -> linear -> relu -> linear -> relu -> linear
      -> MSELoss
      -> loss
```
所以当我们调用 <font color=red>loss.backward()</font> 时，loss就对整张图进行求微分。所有图中 <font color=red>requires_grad=True</font> 的 Tensors 会在他们的 <font color=red>.grad Tensor</font> 累积梯度。

为了说明，让我们顺着反向传播的方向走几步瞧瞧：


In [18]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # ReLU

## 反向传播（Backprop）
为了反向传播错误，我们调用 <font color=red>loss.backward()</font>。你需要清空当前已存在的梯度（gradients），否则梯度（gradients）将被累积到已经存在的梯度。

现在我们调用下 <font color=red>loss.backward()</font>，分别看看 conv1 层偏置项（bias）在反向传播前后的梯度值。

In [19]:
net.zero_grad() # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([ 0.,  0.,  0.,  0.,  0.,  0.])
conv1.bias.grad after backward
tensor([-0.0749,  0.0364, -0.0012,  0.1658, -0.0459, -0.1590])


现在，我们已经知道如何来使用 loss functions。
### 后续阅读：
在神经网络包中包含了很多模块（module）和损失函数（loss functions）。全面的文档详见[这里](https://pytorch.org/docs/stable/nn.html)
### 现在只剩下：
更新网络权重
## 更新权重
在实践中，最简单的更新权重的算法为随机梯度下降法（Stochastic Gradient Descent 【SGD】）：

<font color=red>weight = weight - learning_rate * gradient</font> 

我们可以通过下面的 python 代码简单实现：

In [20]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

然而，当我们使用神经网络时，我们通常想用不同的更新法则，比如 SGD, Nesterov-SGD, Adam, RMSProp 等等。所以 pytorch 提供了一个包： <font color=red>torch.optim</font> 来实现这些方法。运用这个包非常简单：

In [21]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad() # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() # Does the update

<font color=red>**注意**：</font>因为在说明<font color=red>反向传播（Backprop）</font>这一节时，梯度已经累积，所以在这里，我们需要手动用 <font color=red>optimizer.zero_grad()</font> 为 gradients buffers 置0。